In [ ]:
%pip install nbformat openpyxl -q


Note: you may need to restart the kernel to use updated packages.


In [72]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')


In [116]:
# Cargar datasets
files = ['datasets/rm_21_22.xlsx', 'datasets/rm_22_23.xlsx', 'datasets/rm_23_24.xlsx', 'datasets/rm_24_25.xlsx']
dfs = []
for f in files:
    df = pd.read_excel(f)
    df['periodo'] = f.split('_')[1] + '-' + f.split('_')[2].replace('.xlsx', '')
    print(df.columns)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df


Index(['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1',
       'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1',
       'Matches', 'periodo'],
      dtype='object')
Index(['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1',
       'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1',
       'Matches', 'periodo'],
      dtype='object')
Index(['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1',
       'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG',

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,periodo
0,David Alaba,at AUT,DF,29.0,30,30,2642.0,29.4,2.0,3.0,...,0.17,0.07,0.17,0.06,0.07,0.13,0.06,0.13,Matches,21-22
1,Marco Asensio,es ESP,"FW,MF",25.0,31,19,1731.0,19.2,10.0,0.0,...,0.52,0.52,0.52,0.39,0.13,0.52,0.39,0.52,Matches,21-22
2,Gareth Bale,wls WAL,FW,32.0,5,4,280.0,3.1,1.0,0.0,...,0.32,0.32,0.32,0.39,0.03,0.42,0.39,0.42,Matches,21-22
3,Karim Benzema,fr FRA,FW,33.0,32,31,2593.0,28.8,27.0,12.0,...,1.35,0.69,1.11,0.83,0.28,1.10,0.52,0.80,Matches,21-22
4,Antonio Blanco,es ESP,MF,21.0,1,0,31.0,0.3,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,21-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Federico Valverde,uy URU,"MF,DF",26.0,36,34,3032.0,33.7,6.0,4.0,...,0.30,0.18,0.30,0.08,0.09,0.17,0.08,0.17,Matches,24-25
148,Lucas Vázquez,es ESP,DF,33.0,32,25,2189.0,24.3,1.0,5.0,...,0.25,0.04,0.25,0.06,0.19,0.25,0.06,0.25,Matches,24-25
149,Daniel Yáñez,es ESP,FW,17.0,1,0,2.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,24-25
150,Squad Total,NaN,NaN,26.6,38,418,342.0,38.0,78.0,55.0,...,3.50,1.79,3.24,1.98,1.44,3.42,1.70,3.13,NaN,24-25


In [117]:
df.columns

Index(['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1',
       'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1',
       'Matches', 'periodo'],
      dtype='object')

In [118]:
df_nan = df[df.isna().any(axis=1)].copy()
df = df.dropna().drop_duplicates().copy()

df_nan

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,periodo
10,Toni Fuidias,es ESP,GK,20.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,21-22
20,Luis López,es ESP,GK,20.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,21-22
24,Rafa Marín,es ESP,DF,19.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,21-22
29,Álvaro Odriozola,es ESP,DF,25.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,21-22
30,Diego Piñeiro,es ESP,GK,17.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,21-22
36,Squad Total,NaN,NaN,27.7,38,418,342.0,38.0,80.0,59.0,...,3.66,1.89,3.45,1.89,1.33,3.22,1.64,2.97,NaN,21-22
37,Opponent Total,NaN,NaN,27.6,38,418,342.0,38.0,31.0,20.0,...,1.34,0.74,1.26,1.15,0.85,2.00,1.05,1.89,NaN,21-22
43,Lucas Cañizares,es ESP,GK,20.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,22-23
48,Carlos Dotor,es ESP,MF,21.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,22-23
49,Fran González,es ESP,DF,23.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,22-23


In [119]:
df

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,periodo
0,David Alaba,at AUT,DF,29.0,30,30,2642.0,29.4,2.0,3.0,...,0.17,0.07,0.17,0.06,0.07,0.13,0.06,0.13,Matches,21-22
1,Marco Asensio,es ESP,"FW,MF",25.0,31,19,1731.0,19.2,10.0,0.0,...,0.52,0.52,0.52,0.39,0.13,0.52,0.39,0.52,Matches,21-22
2,Gareth Bale,wls WAL,FW,32.0,5,4,280.0,3.1,1.0,0.0,...,0.32,0.32,0.32,0.39,0.03,0.42,0.39,0.42,Matches,21-22
3,Karim Benzema,fr FRA,FW,33.0,32,31,2593.0,28.8,27.0,12.0,...,1.35,0.69,1.11,0.83,0.28,1.10,0.52,0.80,Matches,21-22
4,Antonio Blanco,es ESP,MF,21.0,1,0,31.0,0.3,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,21-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Aurélien Tchouaméni,fr FRA,"DF,MF",24.0,32,31,2689.0,29.9,0.0,0.0,...,0.00,0.00,0.00,0.06,0.02,0.07,0.06,0.07,Matches,24-25
146,Jesús Vallejo,es ESP,DF,27.0,4,1,138.0,1.5,0.0,1.0,...,0.65,0.00,0.65,0.00,0.08,0.08,0.00,0.08,Matches,24-25
147,Federico Valverde,uy URU,"MF,DF",26.0,36,34,3032.0,33.7,6.0,4.0,...,0.30,0.18,0.30,0.08,0.09,0.17,0.08,0.17,Matches,24-25
148,Lucas Vázquez,es ESP,DF,33.0,32,25,2189.0,24.3,1.0,5.0,...,0.25,0.04,0.25,0.06,0.19,0.25,0.06,0.25,Matches,24-25


In [120]:
df["Matches"].unique() # Hay que borrar

array(['Matches'], dtype=object)

In [121]:
df.drop(["Matches", "Nation"], inplace=True, axis=1) # Nation no creo que sea etico, puede encontrar relacion entre razas mejores para el deporte xd
df.columns

Index(['Player', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast',
       'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG',
       'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1',
       'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'periodo'],
      dtype='object')

In [122]:
df.columns

Index(['Player', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast',
       'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG',
       'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1',
       'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'periodo'],
      dtype='object')

In [123]:

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()


In [124]:
cat_cols # Hacerlo por player es mala idea
cat_cols.remove('Player')
cat_cols.remove('periodo')

In [125]:
cat_cols

['Pos']

In [126]:
scaler = StandardScaler()
df_num_scaled = pd.DataFrame(scaler.fit_transform(df[num_cols]), columns=num_cols, index=df.index)

if len(cat_cols) > 0:
    df_cat_dummies = pd.get_dummies(df[cat_cols], drop_first=False, dtype=int)
else:
    df_cat_dummies = pd.DataFrame(index=df.index)

others = df.drop(columns=num_cols + cat_cols, errors='ignore')
df_std = pd.concat([others, df_num_scaled, df_cat_dummies], axis=1)

In [127]:
df_std

,Player,periodo,Age,MP,Starts,Min,90s,Gls,Ast,G+A,...,npxG+xAG.1,Pos_DF,"Pos_DF,MF",Pos_FW,"Pos_FW,DF","Pos_FW,MF",Pos_GK,Pos_MF,"Pos_MF,DF","Pos_MF,FW"
0,David Alaba,21-22,0.656422,0.796556,1.370430,1.562531,1.362587,-0.144840,0.349814,0.021857,...,-0.534696,1,0,0,0,0,0,0,0,0
1,Marco Asensio,21-22,-0.125114,0.877202,0.387295,0.614923,0.420659,1.356225,-0.778831,0.713998,...,0.572035,0,0,0,0,1,0,0,0,0
2,Gareth Bale,21-22,1.242574,-1.219594,-0.953342,-0.894386,-1.066108,-0.332473,-0.778831,-0.531856,...,0.288258,0,0,1,0,0,0,0,0,0
3,Karim Benzema,21-22,1.437958,0.957848,1.459805,1.511562,1.307179,4.545986,3.735750,4.728417,...,1.366611,0,0,1,0,0,0,0,0,0
4,Antonio Blanco,21-22,-0.906650,-1.542178,-1.310846,-1.153392,-1.324676,-0.520106,-0.778831,-0.670284,...,-0.903606,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Aurélien Tchouaméni,24-25,-0.320498,0.957848,1.459805,1.611420,1.408760,-0.520106,-0.778831,-0.670284,...,-0.704962,0,1,0,0,0,0,0,0,0
146,Jesús Vallejo,24-25,0.265654,-1.300240,-1.221470,-1.042092,-1.213861,-0.520106,-0.402616,-0.531856,...,-0.676584,1,0,0,0,0,0,0,0,0
147,Federico Valverde,24-25,0.070270,1.280432,1.727933,1.968204,1.759674,0.605693,0.726029,0.713998,...,-0.421185,0,0,0,0,0,0,0,1,0
148,Lucas Vázquez,24-25,1.437958,0.957848,0.923550,1.091328,0.891623,-0.332473,1.102244,0.160285,...,-0.194163,1,0,0,0,0,0,0,0,0


In [129]:
df_std.columns

Index(['Player', 'periodo', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast',
       'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG',
       'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1',
       'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'Pos_DF',
       'Pos_DF,MF', 'Pos_FW', 'Pos_FW,DF', 'Pos_FW,MF', 'Pos_GK', 'Pos_MF',
       'Pos_MF,DF', 'Pos_MF,FW'],
      dtype='object')

In [131]:
# Top 10 Goleadores
top_goleadores = df.nlargest(10, 'Gls')[['Player', 'periodo', 'Gls', 'Ast', 'G+A', 'MP', 'Min']]
fig = px.bar(top_goleadores, x='Player', y='Gls', color='periodo',
            color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title='Top 10 Goleadores', template='plotly_white', xaxis_tickangle=-45)
fig.show()
top_goleadores


,Player,periodo,Gls,Ast,G+A,MP,Min
134,Kylian Mbappé,24-25,31.0,3.0,34.0,34,2907.0
3,Karim Benzema,21-22,27.0,12.0,39.0,32,2593.0
41,Karim Benzema,22-23,19.0,3.0,22.0,24,2038.0
109,Jude Bellingham,23-24,19.0,6.0,25.0,28,2315.0
17,Vinicius Júnior,21-22,17.0,10.0,27.0,35,269.0
95,Vinicius Júnior,23-24,15.0,5.0,20.0,26,1864.0
131,Vinicius Júnior,24-25,11.0,8.0,19.0,30,2253.0
1,Marco Asensio,21-22,10.0,0.0,10.0,31,1731.0
51,Vinicius Júnior,22-23,10.0,9.0,19.0,33,2823.0
80,Rodrygo,23-24,10.0,5.0,15.0,34,238.0


In [132]:
# Top 10 Asistidores
top_asistidores = df.nlargest(10, 'Ast')[['Player', 'periodo', 'Ast', 'Gls', 'G+A', 'MP']]
fig = px.bar(top_asistidores, x='Player', y='Ast', color='periodo',
            color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title='Top 10 Asistidores', template='plotly_white', xaxis_tickangle=-45)
fig.show()
top_asistidores


,Player,periodo,Ast,Gls,G+A,MP
3,Karim Benzema,21-22,12.0,27.0,39.0,32
17,Vinicius Júnior,21-22,10.0,17.0,27.0,35
51,Vinicius Júnior,22-23,9.0,10.0,19.0,33
27,Luka Modrić,21-22,8.0,2.0,10.0,28
66,Rodrygo,22-23,8.0,9.0,17.0,34
94,Toni Kroos,23-24,8.0,1.0,9.0,33
118,Jude Bellingham,24-25,8.0,9.0,17.0,31
131,Vinicius Júnior,24-25,8.0,11.0,19.0,30
76,Federico Valverde,23-24,7.0,2.0,9.0,37
40,Marco Asensio,22-23,6.0,9.0,15.0,31


In [104]:
df.columns

Index(['Player', 'periodo', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast',
       'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG',
       'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1',
       'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'Pos_DF',
       'Pos_DF,MF', 'Pos_FW', 'Pos_FW,DF', 'Pos_FW,MF', 'Pos_GK', 'Pos_MF',
       'Pos_MF,DF', 'Pos_MF,FW'],
      dtype='object')

In [133]:
plot_df = df_std[['Player', 'periodo', 'Gls', 'Ast', 'G+A']].copy()
plot_df[['Gls', 'Ast', 'G+A']] = plot_df[['Gls', 'Ast', 'G+A']].apply(pd.to_numeric, errors='coerce')
plot_df = plot_df.dropna(subset=['Gls', 'Ast', 'G+A'])

sizes = plot_df['G+A'] - plot_df['G+A'].min() + 0.1

fig = px.scatter(
    plot_df,
    x='Gls',
    y='Ast',
    hover_data=['Player', 'periodo'],
    size=sizes,
    color='periodo',
    color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b']
)
fig.update_layout(title='Goles vs Asistencias', template='plotly_white')
fig.show()

In [134]:
df.groupby('periodo').sum()["Gls"]

periodo
21-22    80.0
22-23    73.0
23-24    85.0
24-25    78.0
Name: Gls, dtype: float64

In [135]:
# Evolución de Goles por Periodo
goles_periodo = df.groupby('periodo')['Gls'].sum().sort_index()
fig = px.bar(x=goles_periodo.index, y=goles_periodo.values,
            color=goles_periodo.index,
            color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title='Total de Goles por Periodo', xaxis_title='Periodo', 
                 yaxis_title='Goles', template='plotly_white', showlegend=False)
fig.show()

In [152]:
# Distribución de Goles, Asistencias y Partidos
fig = make_subplots(rows=1, cols=3, subplot_titles=['Goles', 'Asistencias', 'Partidos'])

fig.add_trace(go.Histogram(x=df['Gls'], marker_color='#6366f1', name='Gls'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['Ast'], marker_color='#8b5cf6', name='Ast'), row=1, col=2)
fig.add_trace(go.Histogram(x=df['MP'], marker_color='#ec4899', name='MP'), row=1, col=3)

fig.update_layout(height=400, showlegend=False, title='Distribución de Métricas', template='plotly_white')
fig.show()


In [138]:
# Correlación entre métricas de rendimiento
metricas = ['Gls', 'Ast', 'G+A', 'MP', 'Min', 'xG', 'xAG', 'CrdY']
corr = df_std[metricas].corr()
fig = px.imshow(corr, text_auto='.2f', color_continuous_scale='RdBu_r', aspect='auto')
fig.update_layout(title='Correlación entre Métricas de Rendimiento', template='plotly_white', height=600)
fig.show()


In [146]:
df_std.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114 entries, 0 to 149
Data columns (total 41 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      114 non-null    object 
 1   periodo     114 non-null    object 
 2   Age         114 non-null    float64
 3   MP          114 non-null    float64
 4   Starts      114 non-null    float64
 5   Min         114 non-null    float64
 6   90s         114 non-null    float64
 7   Gls         114 non-null    float64
 8   Ast         114 non-null    float64
 9   G+A         114 non-null    float64
 10  G-PK        114 non-null    float64
 11  PK          114 non-null    float64
 12  PKatt       114 non-null    float64
 13  CrdY        114 non-null    float64
 14  CrdR        114 non-null    float64
 15  xG          114 non-null    float64
 16  npxG        114 non-null    float64
 17  xAG         114 non-null    float64
 18  npxG+xAG    114 non-null    float64
 19  PrgC        114 non-null    float6

In [147]:
# Aplicar PCA con todas las componentes
df_pca = df_std.select_dtypes(include=['number'])

pca = PCA()
componentes = pca.fit_transform(df_pca)

# Varianza explicada
varianza_df = pd.DataFrame({
    'Componente': [f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))],
    'Varianza_Explicada': pca.explained_variance_ratio_ * 100,
    'Varianza_Acumulada': np.cumsum(pca.explained_variance_ratio_) * 100
}).round(2)

varianza_df


,Componente,Varianza_Explicada,Varianza_Acumulada
0,PC1,45.38,45.38
1,PC2,18.35,63.73
2,PC3,7.84,71.57
3,PC4,6.01,77.59
4,PC5,4.41,81.99
5,PC6,3.41,85.41
6,PC7,3.04,88.45
7,PC8,2.11,90.55
8,PC9,1.89,92.44
9,PC10,1.62,94.06


In [148]:
# Gráfico de varianza explicada por componente
fig = go.Figure()
fig.add_trace(go.Bar(x=[f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))], 
                     y=pca.explained_variance_ratio_ * 100,
                     name='Varianza Explicada', marker_color='#6366f1'))
fig.add_trace(go.Scatter(x=[f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))], 
                         y=np.cumsum(pca.explained_variance_ratio_) * 100,
                         name='Varianza Acumulada', mode='lines+markers', 
                         marker_color='#ec4899', line=dict(width=3)))
fig.update_layout(title='Varianza Explicada por Componente Principal',
                 xaxis_title='Componente', yaxis_title='Varianza (%)',
                 template='plotly_white', height=500)
fig.show()


PCA para ver cuantas variables se necesitan para explicar la variabilidad de los datos

In [150]:
# Edad vs Goles
fig = px.scatter(df, x='Age', y='Gls', hover_data=['Player', 'periodo'],
                size='MP', color='periodo', trendline='ols',
                color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title=f'Edad vs Goles (Correlación: {df["Age"].corr(df["Gls"]):.2f})',
                 xaxis_title='Edad', yaxis_title='Goles', template='plotly_white')
fig.show()


In [151]:
# Edad vs Asistencias
fig = px.scatter(df, x='Age', y='Ast', hover_data=['Player', 'periodo'],
                size='MP', color='periodo', trendline='ols',
                color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title=f'Edad vs Asistencias (Correlación: {df["Age"].corr(df["Ast"]):.2f})',
                 xaxis_title='Edad', yaxis_title='Asistencias', template='plotly_white')
fig.show()


df # Para datos que no estan estandarizados (hacer graficos)
df_std # Datos estandarizados (ver correlaciones, pca, hacer modelos)